In [1]:
!pip3 install datasets huggingface-hub

In [2]:
from datasets import load_dataset

dataset = load_dataset("agentsea/wave-ui-25k", num_proc=16)

filtered_dataset = dataset['train'].filter(
    lambda x: x['language'] is not None 
    and 'English' in x['language']
    and x['platform'] is not None 
    and 'web' in x['platform']
    , num_proc = 16
)

print("HERE")
dataset_without_hf = filtered_dataset.to_dict()
print(dataset_without_hf)

#def wrap_in_list(example):
#    return {'images': [example['image']]}
#filtered_and_image_wrapped = filtered_dataset.map(wrap_in_list)



print(filtered_dataset.column_names)


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

HERE


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOStream.flush timed out
IOStream.flush timed out


In [28]:
def format_instruction(example):
        instruction = f"""Return the bounding box of the {example['description']}. It's used to {example['purpose']} and if we click it {example['expectation']}."""
        bbox = example['bbox']
        x_res, y_res = example['resolution']
        #bounding boxes are in percentage of the screen
        bbox[0] = bbox[0] / x_res * 100
        bbox[1] = bbox[1] / y_res * 100
        bbox[2] = bbox[2] / x_res * 100
        bbox[3] = bbox[3] / y_res * 100
        bbox = [round(x, 2) for x in bbox]  # Limit to two decimal places
        #print(bbox)
        reply = f"""{str(bbox)}"""
        
        # Create messages format
        messages = [ { "user": instruction, "assistant": reply, "source": "web_scraper" } ]

        
        return messages


In [29]:
print(filtered_dataset.column_names)

filtered_dataset_formatted = filtered_dataset.map(lambda x: {"texts": format_instruction(x)})
# Drop the 'source' column since it's not needed
filtered_dataset_formatted = filtered_dataset_formatted.remove_columns([ 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation'])
# Rename 'prompt' column to 'texts' to match expected format
filtered_dataset_formatted = filtered_dataset_formatted.rename_column('image', 'images')
print(filtered_dataset_formatted.column_names)




['image', 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation']


Map:   0%|          | 0/15937 [00:00<?, ? examples/s]

['images', 'texts']


In [34]:
print(filtered_dataset_formatted[0]['images'])



<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1280x720 at 0x7F438F30FF90>


In [32]:

# Set up HuggingFace API and push
username = "jwaters8978"
repo_name = "web_scraper_dataset"

api = HfApi()
api.create_repo(repo_id=f"{username}/{repo_name}", repo_type="dataset", exist_ok=True)
filtered_dataset_formatted.push_to_hub(f"{username}/{repo_name}")

# Print info
print(f"\nPushed to {repo_name}:")
print(f"Number of rows: {len(filtered_dataset)}")
print("\nSample of filtered data:")
print(filtered_dataset.shuffle(seed=42).select(range(5)))

['images', 'texts']


Uploading the dataset shards:   0%|          | 0/13 [00:00<?, ?it/s]

Map:   0%|          | 0/1226 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Map:   0%|          | 0/1226 [00:00<?, ? examples/s]

...

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Map:   0%|          | 0/1225 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.



Pushed to web_scraper_dataset:
Number of rows: 15937

Sample of filtered data:
Dataset({
    features: ['image', 'instruction', 'bbox', 'resolution', 'source', 'platform', 'name', 'description', 'type', 'OCR', 'language', 'purpose', 'expectation'],
    num_rows: 5
})
